In [1]:
import numpy as np
import pandas as pd
import json
from utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.DataFrame(columns=['postText','truthClass'])
with open('data/clickbait_data.txt') as data:
    for sentence in data:
        positive_case = {'truthClass': 1, 'postText': sentence}
        df = df.append(positive_case, ignore_index = True)
df.head()

,postText,truthClass
0,Should I Get Bings\n,1
1,Which TV Female Friend Group Do You Belong In\n,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [3]:
with open('data/non_clickbait_data.txt', encoding='utf-8') as data:
    for sentence in data:
        negative_case = {'truthClass': 0, 'postText': sentence}
        df = df.append(negative_case, ignore_index = True)
df = df.sample(frac=1)
df.head()

,postText,truthClass
7259,David Beckham Immortalized His Daughters Drawi...,1
24285,Wikinews interviews spokesman for Greek far-le...,0
10093,14 Pictures That Sum Up Christmas If You're Th...,1
17258,"Despite Torture Video, U.S. and Emirates Sign ...",0
5688,Would You Be A Better Date Than My New Man\n,1


In [4]:
print(df[df['truthClass'] == 0].shape)
print(df[df['truthClass'] == 1].shape)

(16001, 2)
(15999, 2)


In [5]:
import string
import re
def cleanText(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\w\s]', '', text) 
    text = text[:-1]
    return text
df['postText'].dropna(inplace=True)
df['postText'] = df['postText'].apply(cleanText)
df.head()

,postText,truthClass
7259,david beckham immortalized his daughters drawi...,1
24285,wikinews interviews spokesman for greek farlef...,0
10093,14 pictures that sum up christmas if youre the...,1
17258,despite torture video us and emirates sign key...,0
5688,would you be a better date than my new man,1


In [6]:
maxLen = 0
for i in range(len(df)):
    sentence = df["postText"][i]
    if len(sentence.split()) > maxLen:
        maxLen = len(sentence.split())
        maxstr = sentence
maxLen

26

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.50d.txt')

In [8]:
train, test = train_test_split(df, test_size=0.1)
X_train, Y_train = np.array(train["postText"].tolist()), np.array(train["truthClass"].tolist())
X_test, Y_test = np.array(test["postText"].tolist()), np.array(test["truthClass"].tolist())
print(X_train.shape)
print(X_test.shape)

(28800,)
(3200,)


In [9]:
def sentences_to_indices(X, word_to_index, max_len):   
    m = X.shape[0]  # number of training examples
    # Initialize X_indices as a numpy matrix of zeros and the correct shape 
    X_indices = np.zeros((m,max_len))
    for i in range(m):          
        # Convert the ith training sentence in lower case and split is into words
        sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            if w in word_to_index.keys():
                X_indices[i, j] = word_to_index[w]
                j = j + 1
    return X_indices

In [10]:
Indices = sentences_to_indices(X_train,word_to_index, maxLen)
print("X_Train_indices =\n", Indices.shape)

X_Train_indices =
 (28800, 26)


In [11]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1        
    # define dimensionality of your GloVe word vectors (= 50)
    emb_dim = word_to_vec_map["happy"].shape[0]      
    # Initialize the embedding matrix as a numpy array of zeros.
    # See instructions above to choose the correct shape.
    emb_matrix = np.zeros((vocab_len,emb_dim))
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = Embedding(vocab_len,emb_dim,trainable = False)
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,)) 
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [12]:
def ClickBait_LSTM(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    X = LSTM(128, return_sequences=True)(embeddings)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 2 units
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)  
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X) 
    return model

In [13]:
model = ClickBait_LSTM((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 26, 50)            20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 26, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 26, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 2)
X_train_indices.shape

(28800, 26)

In [16]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 64, shuffle=True)

Epoch 1/10
450/450 [==============================] - 8s 17ms/step - loss: 0.1480 - accuracy: 0.9426
Epoch 2/10
450/450 [==============================] - 9s 21ms/step - loss: 0.0955 - accuracy: 0.9650
Epoch 3/10
450/450 [==============================] - 12s 26ms/step - loss: 0.0832 - accuracy: 0.9704
Epoch 4/10
450/450 [==============================] - 12s 27ms/step - loss: 0.0722 - accuracy: 0.9747
Epoch 5/10
450/450 [==============================] - 12s 27ms/step - loss: 0.0663 - accuracy: 0.9772
Epoch 6/10
450/450 [==============================] - 12s 27ms/step - loss: 0.0581 - accuracy: 0.9794
Epoch 7/10
450/450 [==============================] - 12s 27ms/step - loss: 0.0522 - accuracy: 0.9816
Epoch 8/10
450/450 [==============================] - 12s 27ms/step - loss: 0.0473 - accuracy: 0.9847
Epoch 9/10
450/450 [==============================] - 12s 28ms/step - loss: 0.0407 - accuracy: 0.9861
Epoch 10/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0363 

In [17]:
def onehot_to_binary(data):
    binary = []
    for i in range(len(data)):
        binary.append(1) if data[i][1] > data[i][0] else binary.append(0)
    return binary

In [18]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error 
y_train_pred_oh = model.predict(X_train_indices)
y_train_pred_binary = onehot_to_binary(y_train_pred_oh)
Y_train_binary = onehot_to_binary(Y_train_oh)

print("Training Error")
print('Accuracy %s' % accuracy_score(Y_train_binary, y_train_pred_binary))
print('Precision %s' % precision_score(Y_train_binary, y_train_pred_binary))
print('Recall %s' % recall_score(Y_train_binary, y_train_pred_binary))
print('F1 score: %s' % f1_score(Y_train_binary, y_train_pred_binary))

Training Error
Accuracy 0.9899305555555555
Precision 0.982698488682213
Recall 0.9974318039841744
F1 score: 0.9900103341370996


In [19]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
y_pred_onehot = model.predict(X_test_indices)
y_pred_binary = onehot_to_binary(y_pred_onehot)

print("Testing Error")
print('Accuracy %s' % accuracy_score(Y_test, y_pred_binary))
print('Precision %s' % precision_score(Y_test, y_pred_binary))
print('Recall %s' % recall_score(Y_test, y_pred_binary))
print('F1 score: %s' % f1_score(Y_test, y_pred_binary))

Testing Error
Accuracy 0.9640625
Precision 0.9522351500306185
Recall 0.9767587939698492
F1 score: 0.9643410852713178


In [20]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1608
           1       0.95      0.98      0.96      1592

    accuracy                           0.96      3200
   macro avg       0.96      0.96      0.96      3200
weighted avg       0.96      0.96      0.96      3200



In [21]:
def test(test_string):
    test_string = cleanText(test_string)
    test = np.array([test_string])
    test_indices = sentences_to_indices(test, word_to_index, max_len = maxLen)
    y_pred_onehot = model.predict(test_indices)
    y_pred_binary = onehot_to_binary(y_pred_onehot)
    return y_pred_binary == [1]

test_string = "first comes tinder then comes marriage"
test(test_string)

False